In [25]:
pip install lxml

     |████████████████████████████████| 5.5MB 8.8MB/s eta 0:00:01             | 2.9MB 8.8MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [85]:
import requests
import lxml.html as lh
import pandas as pd

In [86]:
url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

con = lh.fromstring(url.content)

tr_elements = con.xpath('//tr')

In [87]:
[len(T) for T in tr_ele[:12]]
tr_elements = con.xpath('//tr')
col=[]
i=0
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print ('%d:"%s"'%(i,name))
    col.append((name,[]))

1:"Postal Code
"
2:"Borough
"
3:"Neighborhood
"


In [90]:
for j in range(1,len(tr_elements)):
    
    T=tr_elements[j]
    
    
    if len(T)!=3:
        break
    
   
    i=0
    
    
    for t in T.iterchildren():
        data=t.text_content() 
        
        if i>0:
        
            try:
                data=int(data)
            except:
                pass
       
        col[i][1].append(data)
        
        i+=1

In [91]:
[len(C) for (title,C) in col]

[362, 362, 362]

In [92]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

In [93]:
df.head()

,Postal Code\n,Borough\n,Neighborhood\n
0,M1A\n,Not assigned\n,\n
1,M2A\n,Not assigned\n,\n
2,M3A\n,North York\n,Parkwoods\n
3,M4A\n,North York\n,Victoria Village\n
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n"


In [94]:
df.columns = ['Postcode', 'Borough','Neighbourhood']

cols = df.columns.tolist()
cols

cols = cols[-1:] + cols[:-1]

df = df[cols]

df.head()

,Neighbourhood,Postcode,Borough
0,\n,M1A\n,Not assigned\n
1,\n,M2A\n,Not assigned\n
2,Parkwoods\n,M3A\n,North York\n
3,Victoria Village\n,M4A\n,North York\n
4,"Regent Park, Harbourfront\n",M5A\n,Downtown Toronto\n


In [95]:

df.drop(df.index[df['Borough'] == 'Not assigned\n'], inplace = True)

# Reset the index and dropping the previous index
df = df.reset_index(drop=True)

df.head(10)

,Neighbourhood,Postcode,Borough
0,Parkwoods\n,M3A\n,North York\n
1,Victoria Village\n,M4A\n,North York\n
2,"Regent Park, Harbourfront\n",M5A\n,Downtown Toronto\n
3,"Lawrence Manor, Lawrence Heights\n",M6A\n,North York\n
4,"Queen's Park, Ontario Provincial Government\n",M7A\n,Downtown Toronto\n
5,"Islington Avenue, Humber Valley Village\n",M9A\n,Etobicoke\n
6,"Malvern, Rouge\n",M1B\n,Scarborough\n
7,Don Mills\n,M3B\n,North York\n
8,"Parkview Hill, Woodbine Gardens\n",M4B\n,East York\n
9,"Garden District, Ryerson\n",M5B\n,Downtown Toronto\n


In [96]:
df = df.replace('\n',' ', regex=True)
df.head()


,Neighbourhood,Postcode,Borough
0,Parkwoods,M3A,North York
1,Victoria Village,M4A,North York
2,"Regent Park, Harbourfront",M5A,Downtown Toronto
3,"Lawrence Manor, Lawrence Heights",M6A,North York
4,"Queen's Park, Ontario Provincial Government",M7A,Downtown Toronto


In [97]:
df = df.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(','.join).reset_index()
df.columns = ['Postcode','Borough','Neighbourhood']
df.head(10)

,Postcode,Borough,Neighbourhood
0,,Canadian postal codes,","
1,M1B,Scarborough,"Malvern, Rouge ,Malvern, Rouge"
2,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek ,Rouge ..."
3,M1E,Scarborough,"Guildwood, Morningside, West Hill ,Guildwood, ..."
4,M1G,Scarborough,"Woburn ,Woburn"
5,M1H,Scarborough,"Cedarbrae ,Cedarbrae"
6,M1J,Scarborough,"Scarborough Village ,Scarborough Village"
7,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park ,K..."
8,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge ,Golden Mile, ..."
9,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village Wes..."


In [98]:
df['Neighbourhood'] = df['Neighbourhood'].str.strip()
df.loc[df['Neighbourhood'] == 'Not assigned', 'Neighbourhood'] = df['Borough']
df[df['Borough'] == "Queen's Park"]

,Postcode,Borough,Neighbourhood


In [99]:
df.shape

(104, 3)

In [101]:
df.to_csv(r'df_can.csv')